# Obtener con web scraping el listado de ponentes del DES 2021

En la página oficial de Digital Enterprise Show (DES 2021) https://www.des-madrid.com/es/ se puede encontrar información sobre los ponentes que participan en las diferentes charlas del evento. La información se encuentra ordenada alfabéticamente por el nombre del speaker en un total de 8 páginas. A página con la información específica de cada uno de los profesionales se puede acceder desde el listado inicial.

El objetivo de este notebook es agrupar en un fichero CSV la información básica sobre estos profesionales: nombre, cargo, compañía, biografía y enlaces a sus redes sociales profesionales (linkedin y Twitter).

In [68]:
# Importar librerías
import pandas as pd
import numpy as np
import urllib
from bs4 import BeautifulSoup
from time import time


En primer lugar definimos las funciones que vamos a utilizar más adelante: 
* fx_extrae_detalles: Extrae los datos sobre la bio y rrss de los ponentes
* fx_info_speaker: Crea las fichas con toda la info de los ponentes que hay en cada página del listado principal

In [70]:
'''
fx_extrae_detalles

Esta función recoge los datos de biografía y enlaces a RRSS del ponente
Parámetros: url personalizada del ponente y el headers que usamos en todo el programa para las llamadas a BS
Return: Datos de biografía, url de linkedin y url de Twitter del ponente
'''

#def fx_extrae_detalles

def fx_extrae_detalles(url_mas_info,headers):
    
    req = urllib.request.Request(url_mas_info, headers=headers)
    response = urllib.request.urlopen(req)
    soup3 = BeautifulSoup(response.read(), 'html.parser')

    speaker_rrss_container=soup3.find_all("ul",class_="list-social-profiles mt-2")
    for container in speaker_rrss_container:
        try:
            tag_rrss=container("a")

            url_linkedin=tag_rrss[0].get("href")
            if len(tag_rrss)>1:
                url_TW=tag_rrss[1].get("href") 
            else:
                url_TW="no disponible"
            
        except:
            url_linkedin="no disponible"
            url_TW="no disponible"

    speaker_bio_container=soup3("div",class_="module-agenda-text-content")

    speaker_bio_tag=speaker_bio_container[3]("p")
    speaker_bio=""
    for tag in speaker_bio_tag:
        if (len(tag.attrs)) == 0: # Con esta condición filtro solo el texto que me interesa
            a= tag.contents[0] # Extraigo el contenido de la etiqueta
            speaker_bio = speaker_bio + a # Voy concatenando cada contenido a la variable bio

    detalles=[speaker_bio, url_linkedin,url_TW]
    return detalles

In [71]:
'''
fx_info_speaker()

Función para obtener nombre, empresa y cargo de cada uno de los speakers de la página principal. 
Parámetros de entrada: resultado del soup de la página de primer nivel, columnas del dataframe y headers
Return: Dataframe con las fichas de los ponentes de esa página
Esta función llama a fx_extrae_detalles() para obtener el resto de datos para la ficha usando como parámetro de entrada la url de la página personalizada del ponente

''' 

def fx_info_speaker(soup,column_names,headers):

    # Buscamos los contenedores con la info de los speakers
    speaker_container=soup.find_all("div",class_="entity-speakers-box")

    df_ficha_speakers=pd.DataFrame(columns=column_names)


    for container in speaker_container:
        

        tag_nombres=container("h4")
        nombre=tag_nombres[0].contents[0]
        try:
            tag_companys=container.find_all("p", class_="speaker-business-company")
            company=str(tag_companys[0].contents[0]).replace("<strong>","").replace("</strong>","")
        except:
            company="Freelance"
        try:
            tag_position=container.find_all("p", class_="speaker-business-position")
            position=str(tag_position[0].contents[0])
        except:
            company="sin cargo"
        
        tag_mas_info=container("a")
        try:
            url_mas_info=tag_mas_info[0].get("href")
            detalles=fx_extrae_detalles(url_mas_info,headers)
        
            
        except:
            url_mas_info="sin información adicional"
            detalles=["no disponible","no disponible","no disponible"]
        ficha = [nombre, company, position,url_mas_info]
        
        # Llamo a la función fx_extrae_detalles() para ir a la pág. del siguiente nivel a leer la bio y RRSS
        ficha.extend(detalles)
        df_ficha_speakers.loc[len(df_ficha_speakers)] = ficha
        
    #print(df_ficha_speakers.info())

    return (df_ficha_speakers)
  

In [72]:
start_time = time()

'''
La página principal se divide en 7 páginas de listados de ponentes en DES 2021.
La primera es: https://www.des-madrid.com/agenda-2021-speakers/
El resto son esa url seguida de "page/nº/"
'''

# Crear la lista de urls con el listado de ponentes
url_principal="https://www.des-madrid.com/agenda-2021-speakers/"
lista_urls=[url_principal]
for i in range(2,8):
    siguiente=url_principal+"page/"+str(i)+"/"
    lista_urls.append(siguiente)
    
# Estructura del dataframe
column_names=("nombre","company","cargo","más info","speaker_bio", "url_linkedin","url_TW")
df_ficha_speakers=pd.DataFrame(columns=column_names)
df_ficha_speakers_parcial=pd.DataFrame(columns=column_names)

# headers que voy a usar 
headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:80.0) Gecko/20100101 Firefox/80.0'}


#recorremos cada una de las páginas y en ellas extraeremos la información de cada uno de los ponentes
#llamando a la función que lee los datos del ponente
for url in lista_urls:
    
    req = urllib.request.Request(url, headers=headers)
    response = urllib.request.urlopen(req)
    soup = BeautifulSoup(response.read(), 'html.parser')
    
    # llamada a la función que extrae la info y que devuelve un dataframe parcial con la info de los ponente de esa página
    df_ficha_speakers_parcial=fx_info_speaker(soup,column_names,headers)
    
    # añadir las fichas de esa página al df total
    df_ficha_speakers=df_ficha_speakers.append(df_ficha_speakers_parcial)
    
# guardamos en un fichero CSV el listado completo
df_ficha_speakers.to_csv("ponentes_DES_2021.csv")

elapsed_time = time() - start_time
print("Tiempo de ejecución empleado: %.10f seconds." % elapsed_time)

https://www.des-madrid.com/agenda-2021-speakers/
https://www.des-madrid.com/agenda-2021-speakers/page/2/
https://www.des-madrid.com/agenda-2021-speakers/page/3/
https://www.des-madrid.com/agenda-2021-speakers/page/4/
https://www.des-madrid.com/agenda-2021-speakers/page/5/
https://www.des-madrid.com/agenda-2021-speakers/page/6/
https://www.des-madrid.com/agenda-2021-speakers/page/7/
Elapsed time: 590.7714700699 seconds.


In [73]:
# Esta es la información recogida. 413 ponentes en total
df_ficha_speakers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 413 entries, 0 to 52
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   nombre        413 non-null    object
 1   company       413 non-null    object
 2   cargo         413 non-null    object
 3   más info      413 non-null    object
 4   speaker_bio   413 non-null    object
 5   url_linkedin  413 non-null    object
 6   url_TW        413 non-null    object
dtypes: object(7)
memory usage: 25.8+ KB


In [76]:
# ejemplo de un par de fichas
df_ficha_speakers.tail(2)

,nombre,company,cargo,más info,speaker_bio,url_linkedin,url_TW
51,Xavier Idevik,DonDominio,Marketing Director,https://www.des-madrid.com/agenda-2021-speaker...,"Mi nombre es Xavier Idevik, llevo trabajando e...",https://www.linkedin.com/in/xavieridevik/,TW no disponible
52,Yvonne Flores,FYCMA- Foro Transfiere,Head of TradeFairs,https://www.des-madrid.com/agenda-2021-speaker...,Head of Trade Fairs at Fycma and graduated fro...,https://www.linkedin.com/in/yvonne-flores-b6a2...,TW no disponible
